# 🥂 Whisky Meta Data Crawling

### 필요 라이브러리
- bs4
- requests
- time
- pandas
- tqdm
- re
- json

### flavor에 대한 정보를 갖고 있는 위스키만 사용합니다.

In [1]:
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import os
import time as tt
import pandas as pd
from tqdm.notebook import tqdm
import re

import json

In [2]:
whiskies = pd.read_csv(os.getcwd() + "\\dataset\\whisky_basic\\save_1_600.csv", index_col = 0)
whiskies

,link,image,name,avr_rating,category,location
0,/spirits/hibiki-21-year,https://ip-distiller.imgix.net/images/spirits/...,Hibiki 21 Year,4.52,Blended,Japan
1,/spirits/highland-park-18,https://ip-distiller.imgix.net/images/spirits/...,Highland Park 18 Year,4.47,Peated Single Malt,"Islands, Scotland"
2,/spirits/michter-s-20-year-kentucky-straight-b...,https://ip-distiller.imgix.net/images/spirits/...,Michter's 20 Year Kentucky Straight Bourbon (2...,4.61,Bourbon,"Kentucky, USA"
3,/spirits/george-t-stagg-bourbon-fall-2019,https://ip-distiller.imgix.net/images/spirits/...,George T. Stagg Bourbon (Fall 2019),4.61,Bourbon,"Kentucky, USA"
4,/spirits/bowmore-mizunara-cask-finish,https://ip-distiller.imgix.net/images/spirits/...,Bowmore Mizunara Cask Finish,4.08,Peated Single Malt,"Islay, Scotland"
...,...,...,...,...,...,...
3652,/spirits/westland-single-cask-no-1766,https://ip-distiller.imgix.net/images/spirits/...,Westland Single Cask No. 1766,5.00,American Single Malt,USA
3653,/spirits/barrell-whiskey-infinite-barrel-proje...,https://ip-distiller.imgix.net/images/spirits/...,"Barrell Whiskey Infinite Barrel Project (Dec, ...",NaN,Bourbon,USA
3654,/spirits/fercullen-single-malt-aged-20-years,https://ip-distiller.imgix.net/images/spirits/...,Fercullen Single Malt aged 20 years,NaN,Single Malt,Ireland
3655,/spirits/knob-creek-single-barrel-liquor-barn-...,https://ip-distiller.imgix.net/images/spirits/...,"Knob Creek Single Barrel ""Liquor Barn 2021 #6""",4.00,Bourbon,USA


## whisky 정보 요약
---
- 159개의 위스키 리뷰 데이터 부재
- 약 3500개의 위스키만 flavor data 존재

In [3]:
print(whiskies['link'].isnull().sum())
print(whiskies['image'].isnull().sum())
print(whiskies['name'].isnull().sum())
print(whiskies['avr_rating'].isnull().sum())
print(whiskies['category'].isnull().sum())
print(whiskies['location'].isnull().sum())
whiskies.info()

0
0
0
159
0
0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3657 entries, 0 to 3656
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   link        3657 non-null   object 
 1   image       3657 non-null   object 
 2   name        3657 non-null   object 
 3   avr_rating  3498 non-null   float64
 4   category    3657 non-null   object 
 5   location    3657 non-null   object 
dtypes: float64(1), object(5)
memory usage: 200.0+ KB


In [4]:
chrome_driver = os.getcwd()+"\\chromedriver.exe"
base_url = "https://distiller.com"

In [5]:
## cost extract regular expression
number_regex = '\d+'

#### whisky name list

In [6]:
names = whiskies.name
names

0                                          Hibiki 21 Year
1                                   Highland Park 18 Year
2       Michter's 20 Year Kentucky Straight Bourbon (2...
3                     George T. Stagg Bourbon (Fall 2019)
4                            Bowmore Mizunara Cask Finish
                              ...                        
3652                        Westland Single Cask No. 1766
3653    Barrell Whiskey Infinite Barrel Project (Dec, ...
3654                  Fercullen Single Malt aged 20 years
3655       Knob Creek Single Barrel "Liquor Barn 2021 #6"
3656               Knob Creek Single Barrel "Fred Noe #6"
Name: name, Length: 3657, dtype: object

## 크롤링은 할당량

- 다운 : [0:700]
- 지은 : [700:1400]
- 은성 : [1400:2100]
- 예진 : [2100:2800]
- 준현 : [2800:]

### ✨ start와 end 값만 수정하시면 됩니다!!

In [7]:
## 각자 시작번호부터 끝 번호를 작성해주세요.
start = 2800
# end = 

In [8]:
## Total reviewers
total_ratings = []
## Cost
cost_ranks = []
## ABV
abvs = []
## Cask Type
cask_types = []
## Flavor
smoky = []
peaty = []
spicy = []
herbal = []
oily = []
full_bodied = []
rich = []
sweet = []
briny =[]
salty = []
vanilla =[]
tart = []
fruity = []
floral = []
end = 0
 
for i, link in enumerate(tqdm(whiskies["link"][start:])):
    print("link : {} name : {}".format(link, names[i+start]))
    end = i+start

    url = base_url + link
    req = requests.get(url)
    tt.sleep(1.5)
    soup = BeautifulSoup(req.text, "lxml")
    
    ## Total Reviewrs - 리뷰어 수
    try:
        total_reviewer = soup.select_one('span.count').get_text()
    except AttributeError:
        total_reviewer = 0
    

    ## cost ranking - 가격대
    cost = soup.find(class_=lambda x: x and x.startswith('spirit-cost'))
    if(cost ==  None):
        print("{}번째 가격 정보 손실".format(i+start))
        cost_rank = None
    elif cost.has_attr('class'):
        cost_rank = re.search(number_regex, cost['class'][1]).group(0)
    
    ## abv - 도수
    try:
        abv = soup.select_one(".abv > .value").get_text()
    except AttributeError:
        abv = None
    
    ## cask_type
    try:
        cask = soup.select_one(".cask-type > .value").get_text()
    except AttributeError:
        cask = None
    
    ## flavor 
    flavor_summary = soup.select_one("canvas.js-flavor-profile-chart")
    if(flavor_summary == None):
        print("{}번째  맛 정보 손실".format(i+start, link, names[i+start]))
        break
    elif(flavor_summary.has_attr("data-flavors")):
        flavor = json.loads(flavor_summary["data-flavors"])
        smoky.append(flavor["smoky"])
        peaty.append(flavor["peaty"])
        spicy.append(flavor["spicy"])
        herbal.append(flavor["herbal"])
        oily.append(flavor["oily"])
        full_bodied.append(flavor["full_bodied"])
        rich.append(flavor["rich"])
        sweet.append(flavor["sweet"])
        briny.append(flavor["briny"])
        salty.append(flavor["salty"])
        vanilla.append(flavor["vanilla"])
        tart.append(flavor["tart"])
        fruity.append(flavor["fruity"])
        floral.append(flavor["floral"])
        
        total_ratings.append(total_reviewer)
        cost_ranks.append(cost_rank)
        abvs.append(abv)
        cask_types.append(cask)
        
        
    

  0%|          | 0/857 [00:00<?, ?it/s]

link : /spirits/cooper-river-single-run-series-whiskey-st-benjamin-s-ipa name : Cooper River Single Run Series Whiskey-St. Benjamin's IPA
link : /spirits/nant-pinot-noir-single-cask-single-malt name : Nant Pinot Noir Single Cask Single Malt
link : /spirits/tualatin-valley-fifty-fifty-american-whiskey name : Tualatin Valley Fifty/Fifty American Whiskey
link : /spirits/nikka-all-malt-whisky name : Nikka All Malt Whisky
link : /spirits/the-macallan-fine-oak-18-year name : Macallan Fine Oak 18 Year
link : /spirits/old-taylor-straight-bourbon name : Old Taylor Straight Bourbon
link : /spirits/glenfiddich-rich-oak name : Glenfiddich 14 Year Rich Oak
link : /spirits/abraham-bowman-limited-edition-high-rye-bourbon name : Abraham Bowman High Rye Bourbon
link : /spirits/chattanooga-whiskey-co-1816-reserve name : Chattanooga Whiskey Co. 1816 Reserve
link : /spirits/hellyers-road-peated-single-malt name : Hellyers Road Peated Single Malt
link : /spirits/duke-kentucky-straight-bourbon name : Duke K

link : /spirits/eden-mill-hip-flask-series-no-5 name : Eden Mill Hip Flask Series No. 5
link : /spirits/four-roses-bourbon name : Four Roses Bourbon
link : /spirits/prizefight-irish-whiskey name : Prizefight Irish Whiskey
link : /spirits/deadwood-rye name : Deadwood American Rye
link : /spirits/deadwood-bourbon name : Deadwood Straight Bourbon
link : /spirits/glen-scotia-1832-campbeltown name : Glen Scotia 1832 Campbeltown
link : /spirits/bird-dog-small-batch-bourbon name : Bird Dog Small Batch Bourbon (Treeing Walker Coonhound)
link : /spirits/highland-park-full-volume name : Highland Park Full Volume
link : /spirits/lost-distillery-dalaruan name : Lost Distillery Dalaruan
link : /spirits/buffalo-trace-experimental-collection-organic-6-grain-whiskey name : Buffalo Trace Experimental Collection Organic 6 Grain Whiskey
link : /spirits/liquormen-s-ol-dirty-canadian-whisky name : Liquormen's Ol' Dirty Canadian Whisky
link : /spirits/copper-run-small-batch-spirit-whiskey name : Copper Run 

link : /spirits/ri-1-straight-rye-whiskey name : (ri)1 Straight Rye Whiskey
link : /spirits/battle-cry-american-single-malt name : Sons of Liberty Battle Cry American Single Malt
link : /spirits/redemption-high-rye-bourbon name : Redemption High Rye Bourbon
link : /spirits/usquaebach-old-rare-whisky-stone-flagon name : Usquaebach Old Rare Whisky Stone Flagon
link : /spirits/connemara-turf-mor name : Connemara Turf Mór
link : /spirits/the-macallan-gold-1824-series name : Macallan Gold 1824 Series
link : /spirits/oyo-wheat-whiskey name : Oyo Whiskey
link : /spirits/five-fathers-rye name : Five Fathers Rye
link : /spirits/mortlach-25-year name : Mortlach 25 Year
link : /spirits/journeyman-distillery-featherbone-bourbon name : Journeyman Distillery Featherbone Organic Bourbon
link : /spirits/journeyman-distillery-ravenswood-rye name : Journeyman Distillery Last Feather Organic Rye
link : /spirits/jack-daniel-s-rested-tennessee-rye name : Jack Daniel's Rested Tennessee Rye
link : /spirits/t

link : /spirits/highland-reserve-blended-malt name : Highland Reserve Blended Malt
link : /spirits/manatawny-still-works-small-batch-whiskey-batch-5 name : Manatawny Still Works Small Batch Whiskey (Batch #5)
link : /spirits/new-zealand-whisky-collection-dunedin-doublewood-10-year-blended-whisky name : Dunedin Doublewood 10 Year
link : /spirits/barrell-bourbon-batch-005 name : Barrell Bourbon Batch 005
link : /spirits/coppersea-new-york-excelsior-bourbon name : Coppersea Excelsior Bourbon
link : /spirits/defiant-american-single-malt-whisky name : Defiant American Single Malt Whisky
link : /spirits/nant-bourbon-wood-63-single-cask-single-malt name : Nant Bourbon Wood 63% Single Cask Single Malt
link : /spirits/big-house-straight-bourbon-6-year name : Big House Straight Bourbon 6 Year
link : /spirits/iowa-legendary-rye-white-whiskey name : Iowa Legendary Rye White Whiskey
link : /spirits/ardmore-legacy name : Ardmore Legacy
link : /spirits/nikka-pure-malt-white-whisky name : Nikka Pure M

link : /spirits/two-james-rye-dog-white-whiskey name : Two James Rye Dog White Whiskey
link : /spirits/rod-rifle-straight-rye name : Rod & Rifle Straight Rye
link : /spirits/rod-rifle-american-blended-whiskey name : Rod & Rifle American Blended Whiskey
link : /spirits/virgil-kaine-robber-baron-rye-whiskey name : Virgil Kaine Robber Baron Rye Whiskey
link : /spirits/tualatin-valley-oregon-single-malt name : Tualatin Valley Oregon Single Malt
link : /spirits/george-dickel-no-1 name : George Dickel No. 1
link : /spirits/lyon-maryland-free-state-rye name : Lyon Maryland Free State Rye
link : /spirits/pennypacker-bourbon name : PennyPacker Bourbon
link : /spirits/bib-tucker-small-batch-bourbon name : Bib & Tucker 6 Year Small Batch Bourbon
link : /spirits/loch-lomond-single-malt name : Loch Lomond Original
link : /spirits/woodford-reserve-sweet-mash-redux-bourbon name : Woodford Reserve Sweet Mash Redux Bourbon
link : /spirits/1792-sweet-wheat name : 1792 Sweet Wheat
link : /spirits/the-bal

link : /spirits/yellowstone-family-recipe-bourbon-2022-release name : Yellowstone Family Recipe Bourbon (2022 Release)
link : /spirits/jk-williams-american-wheat-whiskey name : JK Williams The Bridge Series: American Wheat Whiskey
link : /spirits/kentucky-owl-the-wiseman-bourbon name : The Wiseman Kentucky Straight Bourbon
link : /spirits/hudson-whiskey-four-part-harmony name : Hudson Whiskey Four Part Harmony
link : /spirits/cascade-moon-edition-no-1 name : Cascade Moon Edition No. 1
link : /spirits/michter-s-10-year-single-barrel-kentucky-straight-bourbon-2020-release name : Michter's 10 Year Single Barrel Kentucky Straight Bourbon (2020 Release)
link : /spirits/corsair-dark-rye name : Corsair Dark Rye
link : /spirits/glenmorangie-the-cadboll-estate-15-year name : Glenmorangie The Cadboll Estate 15 Year
link : /spirits/balblair-15-year name : Balblair 15 Year
link : /spirits/the-lakes-steel-bonnets name : The Lakes Steel Bonnets
link : /spirits/the-singleton-of-glendullan-game-of-thr

link : /spirits/obegley-young-single-malt name : O'Begley Young Single Malt
link : /spirits/wayward-rye-whiskey name : Wayward Rye Whiskey
link : /spirits/usquaebach-15-year-old name : Usquaebach 15 Year
link : /spirits/yellow-rose-blended-whiskey name : Yellow Rose Blended Whiskey
link : /spirits/johnnie-walker-swing name : Johnnie Walker Swing
link : /spirits/johnnie-walker-white-walker name : White Walker by Johnnie Walker
link : /spirits/black-canyon-winter name : Black Canyon Winter
link : /spirits/kingsbarns-new-make-spirit name : Kingsbarns New Make Spirit
link : /spirits/delaware-phoenix-bourbon name : Delaware Phoenix Bourbon
link : /spirits/the-hilhaven-lodge-whiskey name : The Hilhaven Lodge Whiskey
link : /spirits/mack-by-mackmyra name : MACK by Mackmyra
link : /spirits/the-dubliner-irish-whiskey name : The Dubliner Irish Whiskey
link : /spirits/penderyn-celt name : Penderyn Celt
link : /spirits/graveyard-sam-s-white-whiskey name : Graveyard Sam's White Whiskey
link : /spir

link : /spirits/lexington-bourbon name : Lexington Bourbon
link : /spirits/windsor name : Windsor
link : /spirits/canadian-mist name : Canadian Mist
link : /spirits/crown-royal-texas-mesquite name : Crown Royal Texas Mesquite
link : /spirits/teton-moonshine-huckleberry-whiskey name : Teton Moonshine Huckleberry Whiskey
link : /spirits/pink-lightning-strawberry-moonshine name : Pink Lightning Strawberry Moonshine
link : /spirits/j-w-overbey-bourbon name : J.W. Overbey Bourbon
link : /spirits/bowen-s-whiskey name : Bowen's Whiskey
link : /spirits/smokehead name : Smokehead Islay Single Malt
link : /spirits/chattooga-belle-block-and-tackle-whiskey name : Chattooga Belle Block and Tackle Whiskey
link : /spirits/2bar-bourbon name : 2bar Bourbon
link : /spirits/mcclelland-s-single-malt-highland name : McClelland's Single Malt Highland
link : /spirits/mcclelland-s-single-malt-lowland name : McClelland's Single Malt Lowland
link : /spirits/john-b-stetson-kentucky-straight-bourbon-whiskey name 

In [1]:
savePath = os.getcwd()+"\\dataset\\whisky_metadata"
savePath

NameError: name 'os' is not defined

(현재 작업 위치)\\dataset 위치에 생성한 데이터를 csv 파일로 저장합니다.

### 파일 저장
- start, end 기준으로 파일명을 저장합니다.

In [11]:
df = pd.DataFrame({
    "name": names[start:end], "total_rating": total_ratings, "cost_rank": cost_ranks, "abv": abvs, "cask_type": cask_types, "smoky": smoky,
    "peaty": peaty, "spicy": spicy, "herbal": herbal, "oily": oily, "full_bodied": full_bodied, "rich": rich, "sweet": sweet,
    "briny": briny, "salty": salty, "vanilla": vanilla, "tart": tart, "fruity": fruity, "floral": floral
})
df.to_csv(savePath+"\\metadata_{}_{}.csv".format(str(start), str(end)), index=True)